In [1]:
import pandas as pd
import json


In [2]:
DB_DATA_PATH = "data/HungThinhDB10K.csv"

db_data = pd.read_csv(DB_DATA_PATH)


In [17]:
with open('data/intent_alias_data.json', 'r', encoding="utf-8") as fp:
    alias_file = json.load(fp)
    fp.close()


In [18]:

with open('data/create_csv_dict.json', 'r', encoding="utf-8") as fp:
    create_csv_dict = json.load(fp)
    fp.close()

HOUSE = create_csv_dict["HOUSE"]
CITY = create_csv_dict["CITY"]
DISTRICT = create_csv_dict["DISTRICT"]
WARD = create_csv_dict["WARD"]
STREET = create_csv_dict["STREET"]
# HOUSE_NUMBER = create_csv_dict["HOUSE_NUMBER"]
PRICE = create_csv_dict["PRICE"]
AREA = create_csv_dict["AREA"]
POTENTIAL = create_csv_dict["POTENTIAL"]
BATH_ROOM = create_csv_dict["BATH_ROOM"]
BED_ROOM = create_csv_dict["BED_ROOM"]
LIVING_ROOM = create_csv_dict["LIVING_ROOM"]


In [19]:
def preprocess(old_data, old_column_name, new_column_name):
    temp_data = pd.DataFrame(old_data)
    temp_data = temp_data.rename(columns={old_column_name: new_column_name})
    temp_data = temp_data.dropna()
    temp_data = temp_data.drop_duplicates()
    number_column_arr = ['price', 'area', 'bath_room', 'bed_room', 'living_room']

    if old_column_name not in number_column_arr:
        for x in temp_data.index:
            if "http" in temp_data.loc[x, new_column_name]:
                # print(x, temp_data.loc[x, new_column_name])
                temp_data = temp_data.drop(x)

            # if "/" in temp_data.loc[x, new_column_name]:
            #     print(x, temp_data.loc[x, new_column_name])
                # temp_data = temp_data.drop(x)

            # if "." in temp_data.loc[x, new_column_name]:
            #     print(x, temp_data.loc[x, new_column_name])
                # temp_data = temp_data.drop(x)

            # if "<" in temp_data.loc[x, new_column_name]:
            #     print(x, temp_data.loc[x, new_column_name])
            #     temp_data = temp_data.drop(x)

            # if ">" in temp_data.loc[x, new_column_name]:
            #     print(x, temp_data.loc[x, new_column_name])
            #     temp_data = temp_data.drop(x)

            # if "01" in temp_data.loc[x, new_column_name]:
            #     print(x, temp_data.loc[x, new_column_name])
            #     temp_data = temp_data.drop(x)

    return temp_data


In [6]:
def add_alias_to_csv(file_csv_path, new_alias_column_name, alias_list):
    temp_data = pd.read_csv(file_csv_path)
    new_column = pd.DataFrame({new_alias_column_name: alias_list})
    temp_data = temp_data.merge(
        new_column, how="left", left_index=True, right_index=True)
    temp_data.to_csv(file_csv_path, index=False)


In [7]:
def create_alias_column(file_csv_path, new_alias_column_name, alias_file, alias_data):
    alias_list = alias_file[alias_data]
    add_alias_to_csv(file_csv_path, new_alias_column_name, alias_list)


In [8]:
def create_csv_file(db_data, column_name, new_column_name, file_name):
    temp_data_old = db_data[[column_name]]
    new_data = preprocess(temp_data_old, column_name, new_column_name)
    new_data.to_csv(file_name)


In [21]:
# implement house_data.csv
create_csv_file(db_data, HOUSE["column_name"],
                HOUSE["new_column_name"], HOUSE["csv_file_path"])
create_alias_column(HOUSE["csv_file_path"], HOUSE["new_alias_column_name"],
                    alias_file, HOUSE["alias_list"])


In [10]:
# implement city_data.csv
create_csv_file(db_data, CITY["column_name"],
                CITY["new_column_name"], CITY["csv_file_path"])
create_alias_column(CITY["csv_file_path"], CITY["new_alias_column_name"],
                    alias_file, CITY["alias_list"])

In [11]:
# implement district_data.csv
create_csv_file(db_data, DISTRICT["column_name"],
                DISTRICT["new_column_name"], DISTRICT["csv_file_path"])
create_alias_column(DISTRICT["csv_file_path"], DISTRICT["new_alias_column_name"],
                    alias_file, DISTRICT["alias_list"])

In [12]:
# implement ward_data.csv
create_csv_file(db_data, WARD["column_name"],
                WARD["new_column_name"], WARD["csv_file_path"])
create_alias_column(WARD["csv_file_path"], WARD["new_alias_column_name"],
                    alias_file, WARD["alias_list"])


In [13]:
# implement street_data.csv
create_csv_file(db_data, STREET["column_name"],
                STREET["new_column_name"], STREET["csv_file_path"])
create_alias_column(STREET["csv_file_path"], STREET["new_alias_column_name"],
                    alias_file, STREET["alias_list"])


In [14]:
# implement house_number_data.csv
# create_csv_file(db_data, HOUSE_NUMBER["column_name"],
#                 HOUSE_NUMBER["new_column_name"], HOUSE_NUMBER["csv_file_path"])
# create_alias_column(HOUSE_NUMBER["csv_file_path"], HOUSE_NUMBER["new_alias_column_name"],
#                     alias_file, HOUSE_NUMBER["alias_list"])


In [25]:
# implement price_data.csv
create_csv_file(db_data, PRICE["column_name"],
                PRICE["new_column_name"], PRICE["csv_file_path"])
create_alias_column(PRICE["csv_file_path"], PRICE["new_alias_column_name"],
                    alias_file, PRICE["alias_list"])

In [11]:
# implement area_data.csv
create_csv_file(db_data, AREA["column_name"],
                AREA["new_column_name"], AREA["csv_file_path"])
create_alias_column(AREA["csv_file_path"], AREA["new_alias_column_name"],
                    alias_file, AREA["alias_list"])

In [13]:
# implement potential_data.csv
create_csv_file(db_data, POTENTIAL["column_name"],
                POTENTIAL["new_column_name"], POTENTIAL["csv_file_path"])
create_alias_column(POTENTIAL["csv_file_path"], POTENTIAL["new_alias_column_name"],
                    alias_file, POTENTIAL["alias_list"])

In [16]:
# implement bath_room_data.csv
create_csv_file(db_data, BATH_ROOM["column_name"],
                BATH_ROOM["new_column_name"], BATH_ROOM["csv_file_path"])
create_alias_column(BATH_ROOM["csv_file_path"], BATH_ROOM["new_alias_column_name"],
                    alias_file, BATH_ROOM["alias_list"])

In [20]:
# implement bed_room_data.csv
create_csv_file(db_data, BED_ROOM["column_name"],
                BED_ROOM["new_column_name"], BED_ROOM["csv_file_path"])
create_alias_column(BED_ROOM["csv_file_path"], BED_ROOM["new_alias_column_name"],
                    alias_file, BED_ROOM["alias_list"])

In [21]:
# implement living_room_data.csv
create_csv_file(db_data, LIVING_ROOM["column_name"],
                LIVING_ROOM["new_column_name"], LIVING_ROOM["csv_file_path"])
create_alias_column(LIVING_ROOM["csv_file_path"], LIVING_ROOM["new_alias_column_name"],
                    alias_file, LIVING_ROOM["alias_list"])